# Analyzing CIA Factbook Data Using SQL

This project uses SQL in Jupyter Notebook to explore and analyze data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth.

This project will use the [SQLite factbook.db database](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db).

## Connect to DB

In [7]:
import sqlite3 as sql

In [17]:
%%capture
%load_ext sql
%sql sqlite:///data/factbook.db

## Explore Tables in Database
Below returns information on the tables in the factbook database.

In [18]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///data/factbook.db
   sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


### Exploring `facts` Tables

In [19]:
%%sql
SELECT * 
  FROM facts
LIMIT 5;

 * sqlite:///data/factbook.db
   sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### Investigating Outlier Countries
This section will use summary statistics to investigate any outlier countries in the facts table.

In [20]:
%%sql
SELECT MIN(population) AS 'Minimum Population',
       MAX(population) AS 'Maximum Population',
       MIN(population_growth) AS 'Minimum Population Growth',
       MAX(population_growth) AS 'Maximum Population Growth'
  FROM facts;

 * sqlite:///data/factbook.db
   sqlite:///factbook.db
Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,7256490011,0.0,4.02


A few things that stick out from the summary statistics above are:

- There's a country with a population of 0
- There's a country with a population of more that 7.2 billion people.

Below will use sub-queries to zoom in on just these countries without using the specific values.

**Investigating Countries With Minimum Population of 0**

In [21]:
%%sql
SELECT name, population
FROM facts
WHERE population = (SELECT MIN(population)
                      FROM facts
                   );

 * sqlite:///data/factbook.db
   sqlite:///factbook.db
Done.


name,population
Antarctica,0


#### Investigating Countries With the Maximum Population

In [22]:
%%sql
SELECT name, population
FROM facts
WHERE population = (SELECT MAX(population)
                      FROM facts
                   );

 * sqlite:///data/factbook.db
   sqlite:///factbook.db
Done.


name,population
World,7256490011


The results from the investigation above show that:

- The table contains a row for the whole world, which explains the population of over 7.2 billion.
- The table contains a row for Antarctica, which explains the population of 0. According the Factbook page for Antarctica, there are no indigenous inhabitants on Antarctica; only permanent and summer-only staffed research stations.

**Recalculate Summary Statistics**

Next cell will recalculate the summary statistics excluding the row for the whole world.

In [23]:
%%sql
SELECT MIN(population) AS 'Minimum Population',
       MAX(population) AS 'Maximum Population',
       MIN(population_growth) AS 'Minimum Population Growth',
       MAX(population_growth) AS 'Maximum Population Growth'
FROM facts
WHERE name <> 'World'

 * sqlite:///data/factbook.db
   sqlite:///factbook.db
Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,1367485388,0.0,4.02


There's a country with a population close to 1.4 billion!

### Exploring Average Population and Area

In [24]:
%%sql
SELECT AVG(population) AS 'Average Population',
       AVG(area) AS 'Average Area'
  FROM facts
WHERE name <> ('World')

 * sqlite:///data/factbook.db
   sqlite:///factbook.db
Done.


Average Population,Average Area
32242666.56846473,555093.546184739


The average population is around 32 million and the average area is 555 thousand square kilometers.

### Finding Densely Populated Countries
This section will identify countries that are densely populated. Population will be defined as the number of people per square kilometer.

$$population density = \frac{population}{area}$$

In [25]:
%%sql
--Countries with population above average
SELECT name, area, population, population/area AS population_density
  FROM facts
 WHERE name <> 'World'
 ORDER BY population_density DESC
 LIMIT 10;

 * sqlite:///data/factbook.db
   sqlite:///factbook.db
Done.


name,area,population,population_density
Macau,28,592731,21168
Monaco,2,30535,15267
Singapore,697,5674472,8141
Hong Kong,1108,7141106,6445
Gaza Strip,360,1869055,5191
Gibraltar,6,29258,4876
Bahrain,760,1346613,1771
Maldives,298,393253,1319
Malta,316,413965,1310
Bermuda,54,70196,1299


Results show that Macau and Monaco are the most densely populated countries. Although most of these countries have populations lower than the average population, they have small areas.

Monaco has an area of only 2 sqkm!

## Conclusion and Future Work
This project explored the CAI Factbook Dataset.

Future analysis could address the following question:

- Which countries will have highest population in the subsequent year?
- Which countries will have highest change in population in the subsequent year?
- Which countries have a higher death rate than birth rate?

Furthermore follow-up analysis may can include dashboard that provide summary statistics and other useful information from the dataset:

- Create a function that takes queries as input and returns relevant charts that answer the query. For example:
    - Input: top 10 most densely populated countries
    - Output: bar chart showing top 10 most densely populated countries